# K-Nearest Neighbor Lab
Read over the sklearn info on [nearest neighbor learners](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification)




In [1]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from numpy import ndarray
import pandas as pd
from scipy.io import arff
import matplotlib.pyplot as plt
from typing import Tuple
from typing import List
from collections import defaultdict
from tabulate import tabulate
import csv
import math
from enum import Enum
from abc import ABC, abstractmethod

np.set_printoptions(threshold=10000)

In [2]:
def normalize(X: ndarray) -> ndarray:
  return MinMaxScaler().fit_transform(X)

def normalize_continuous_features(X: ndarray, nominal_feature_indexes: set):
  for i in range(X.shape[1]):
    if i not in nominal_feature_indexes:
      X[:, i:i+1] = normalize(X[:, i:i+1])

  return X

def load_glass(flatten=True) -> Tuple[ndarray, ndarray]:
  """
  From: https://archive.ics.uci.edu/dataset/42/glass+identification
  Purpose: Predict glass type

  Features:
    RI	refractive inde (REAL)
    Na	Percent Sodium (REAL)
    Mg	Percent Magnesium (REAL)
    Al	Percent Aluminum (REAL)
    Si	Percent Silicon (REAL)
    K	  Percent Potassium (REAL)
    Ca	Percent Calcium (REAL)
    Ba	Percent Barium (REAL)
    Fe	Percent Iron (REAL)

  Classification:
    Type of glass (CATEGORICAL)
  """
  # load csv file
  with open('glass.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

  # convert to numby array, and extract X and y
  data = np.array(data)
  X = data[:, 1:-1]
  y = data[:, -1:]
  X = X.astype(float)
  y = y.astype(float)
  if flatten:
    y = y.flatten()

  return X, y

def load_magic_telescope() -> Tuple[ndarray, ndarray]:
  """
  Purpose: Classification as gamma or hardon from telestope data

  Features:
    fLength (REAL)
    fWidth (REAL)
    fSize (REAL)
    fConc (REAL)
    fConc1 (REAL)
    fAsym (REAL)
    fM3Long (REAL)
    fM3Trans (REAL)
    fAlpha (REAL)
    fDist (REAL)

  Classifications:
    -> {g, h} (gamma (signal), hadron (background))
  """
  data_arff = arff.loadarff("MagicTelescope.arff")
  data_df = pd.DataFrame(data_arff[0])
  data_np = data_df.to_numpy()
  X = data_np[:, :-1]
  y = data_np[:, -1]

  # convert strings/booleans to numbers
  X = X.astype(float)
  y = np.where(y == b'g', 'g', 'h')

  return X, y

def load_housing() -> Tuple[ndarray, ndarray]:
  """
  Purpose: Regression of home value in $1000's

  Features:
    CRIM    (REAL) per capita crime rate by town
    ZN		  (REAL) proportion of residential land zoned for lots over 25,000 sq.ft.
    INDUS	  (REAL) proportion of non-retail business acres per town
    CHAS		{0,1}  Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
    NOX	  	(REAL) nitric oxides concentration (parts per 10 million)
    RM		  (REAL) average number of rooms per dwelling
    AGE		  (REAL) proportion of owner-occupied units built prior to 1940
    DIS	  	(REAL) weighted distances to five Boston employment centres
    RAD		  (REAL) index of accessibility to radial highways
    TAX		  (REAL) full-value property-tax rate per $10,000
    PTRATIO	(REAL) pupil-teacher ratio by town
    B		    (REAL) 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    LSTAT	  (REAL) % lower status of the population

  Regression:
    -> MEDV (REAL) Median value of owner-occupied homes in $1000's
  """
  data_arff = arff.loadarff("housing.arff")
  data_df = pd.DataFrame(data_arff[0])
  data_np = data_df.to_numpy()
  X = data_np[:, :-1]
  y = data_np[:, -1]

  # convert strings/booleans to numbers
  X = X.astype(float)
  y = y.astype(float)

  return X, y

def load_lymph() -> Tuple[ndarray, ndarray]:
  """
  Purpose: Classification of lymphography

  Features:
    'lymphatics'      { normal, arched, deformed, displaced}
    'block_of_affere' { no, yes}
    'bl_of_lymph_c'   { no, yes}
    'bl_of_lymph_s'   { no, yes}
    'by_pass'         { no, yes}
    'extravasates'    { no, yes}
    'regeneration_of' { no, yes}
    'early_uptake_in' { no, yes}
    'lym_nodes_dimin' integer
    'lym_nodes_enlar' integer
    'changes_in_lym'  { bean, oval, round}
    'defect_in_node'  { no, lacunar, lac_margin, lac_central}
    'changes_in_node' { no, lacunar, lac_margin, lac_central}
    'changes_in_stru' { no, grainy, drop_like, coarse, diluted, reticular, stripped, faint}
    'special_forms'   { no, chalices, vesicles}
    'dislocation_of'  { no, yes}
    'exclusion_of_no' { no, yes}
    'no_of_nodes_in'  integer

  Classifications:
    -> { normal, metastases, malign_lymph, fibrosis }
  """
  data_arff = arff.loadarff("lymph.arff")
  data_df = pd.DataFrame(data_arff[0])
  data_np = data_df.to_numpy()
  X = data_np[:, :-1]
  y = data_np[:, -1]

  # # convert strings/booleans to numbers
  # X = X.astype(float)

  return X, y

## 1 K-Nearest Neighbor (KNN) algorithm

### 1.1 (15%) Basic KNN Classification

Learn the [Glass data set](https://archive.ics.uci.edu/dataset/42/glass+identification) using [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier) with default parameters.
- Randomly split your data into train/test.  Anytime we don't tell you specifics (such as what percentage is train vs test) choose your own reasonable values
- Give typical train and test set accuracies after running with different random splits
- Print the output probabilities for a test set (predict_proba)
- Try it with different p values (Minkowskian exponent) and discuss any differences

In [ ]:
"""
Learn the glass data

p value:
- Used when metric='minkowski' (default)
- Values:
  p < 1 (Closer distances are more important)
  p = 1 (Manhattan Distance)
  p = 2 (Euclidean Distance)
  p > 2 (Farther distances are more important)
"""
X, y = load_glass()

p_values = [0.5, 1, 1.5, 2, 2.5, 5, 1000000]
for p in p_values:
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
  knn = KNeighborsClassifier(p=p)
  knn.fit(X_train, y_train)

  print("p: ", str(p))
  print("   train score: ", str(knn.score(X_train, y_train)))
  print("   test score: ", str(knn.score(X_test, y_test)))
  print("   predict_proba:" , str(knn.predict_proba(X_test)))


p:  0.5
   train score:  0.783625730994152
   test score:  0.7674418604651163
   predict_proba: [[0.  1.  0.  0.  0.  0. ]
 [1.  0.  0.  0.  0.  0. ]
 [0.4 0.6 0.  0.  0.  0. ]
 [0.  1.  0.  0.  0.  0. ]
 [0.8 0.  0.2 0.  0.  0. ]
 [0.4 0.6 0.  0.  0.  0. ]
 [0.  1.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.2]
 [0.  0.  0.  0.  0.  1. ]
 [0.  0.4 0.  0.2 0.2 0.2]
 [0.2 0.6 0.2 0.  0.  0. ]
 [0.2 0.8 0.  0.  0.  0. ]
 [0.6 0.  0.4 0.  0.  0. ]
 [0.6 0.2 0.2 0.  0.  0. ]
 [0.2 0.8 0.  0.  0.  0. ]
 [0.8 0.2 0.  0.  0.  0. ]
 [1.  0.  0.  0.  0.  0. ]
 [0.6 0.2 0.2 0.  0.  0. ]
 [0.  0.4 0.  0.6 0.  0. ]
 [0.8 0.2 0.  0.  0.  0. ]
 [0.6 0.  0.4 0.  0.  0. ]
 [0.2 0.8 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  1. ]
 [0.  0.2 0.  0.8 0.  0. ]
 [0.8 0.  0.2 0.  0.  0. ]
 [0.  1.  0.  0.  0.  0. ]
 [1.  0.  0.  0.  0.  0. ]
 [0.  1.  0.  0.  0.  0. ]
 [0.2 0.6 0.2 0.  0.  0. ]
 [0.  0.6 0.  0.4 0.  0. ]
 [0.8 0.  0.2 0.  0.  0. ]
 [0.  0.2 0.  0.  0.2 0.6]
 [0.  0.  0.  0.  0.  1. ]
 [0.6 0.2 0. 

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_base.py:653: UserWarning: Mind that for 0 < p < 1, Minkowski metrics are not distance metrics. Continuing the execution with `algorithm='brute'`.
  warnings.warn(


Discussion

The results for this trial were quite interesting. The p value is used when the metric hyperparameter is set to 'minkowski', and since this is the default, I didn't have to set it in my model. The p value sets how sensitive the metric is to higher differences in distance. When p is less then 1, it is more sensitive to closer distances. When it is 1, it uses manhattan distance, which can be thought of as the distance to travel somewhere when you can only take street blocks. When p is 2, euclidean distance is used which is "as the crow flies". As it is increased past 2, farther distances (points farther away) become more important than they otherwise would be.

This makes sense to me becuase when you compare Manhattan  to Euclidian distance, Manhattan would yield a further distance for something far away because you can't cut straight to the other point. And if you increased past 2 (Euclidian) it would continue to make the distance less and less for something that is actually far away.

In my trials, it was interesting to find a sweet spot. Of course, if I had done more trials at each p value, I'm sure my results would be a little more accurate, simple because the luck of the train test split is playing a role here. The sweet spot was p=2.5 which is a little past Euclidian distance, and it yielded 86.04% accuracy. Past that, the results dropped. Before 2.5, there was quite a bit of fluctuation, though the next highest was 76.74% for the test set score. It's also interesting that the training set score was pretty constant at around 76% until I increased p to a very high value.

## 2 KNN Classification with normalization and distance weighting

Use the [magic telescope](https://axon.cs.byu.edu/data/uci_class/MagicTelescope.arff) dataset

### 2.1 (5%) - Without Normalization or Distance Weighting
- Do random 80/20 train/test splits each time
- Run with k=3 and *without* distance weighting and *without* normalization
- Show train and test set accuracy

In [ ]:
# Learn magic telescope data
# WOUT/Normalization, WOUT/Distance Weighting
X, y = load_magic_telescope()

avg_train_acc = 0.0
avg_test_acc = 0.0
n_trials = 10
for i in range(n_trials):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
  knn = KNeighborsClassifier(n_neighbors=3, weights='uniform') # NOTE - uniform is default
  knn.fit(X_train, y_train)

  train_acc = knn.score(X_train, y_train)
  test_acc = knn.score(X_test, y_test)

  avg_train_acc += train_acc
  avg_test_acc += test_acc

  print("Trial:")
  print("   train score: ", str(train_acc))
  print("   test score: ", str(test_acc))

print("\nAverages:")
print("   avg train score: ", str(avg_train_acc / n_trials))
print("   avg test score: ", str(avg_test_acc / n_trials))

Trial:
   train score:  0.8903128286014721
   test score:  0.7823343848580442
Trial:
   train score:  0.8858438485804416
   test score:  0.7957413249211357
Trial:
   train score:  0.8851866456361724
   test score:  0.7941640378548895
Trial:
   train score:  0.8870268138801262
   test score:  0.8070452155625657
Trial:
   train score:  0.8860410094637224
   test score:  0.805993690851735
Trial:
   train score:  0.8865667718191378
   test score:  0.7960042060988434
Trial:
   train score:  0.8826235541535226
   test score:  0.8065194532071503
Trial:
   train score:  0.8845951629863302
   test score:  0.7981072555205048
Trial:
   train score:  0.8848580441640379
   test score:  0.7986330178759201
Trial:
   train score:  0.8881440588853838
   test score:  0.7807570977917981

Averages:
   avg train score:  0.8861198738170348
   avg test score:  0.7965299684542588


*Discussion*

In each of the trials, the test accuracy was about 9% lower than the training score, which seems realistic. Here, I did not use normalization. Normalization puts each of the values on the same scale (typically between 0 and 1), and can help results be found more quickly and accurately, and provides overall more standardization to the data.

The other thing to note is that distance weighting was not used. Distance weighting is an add on to plain KNN where points that are closer to the point q have a higher weight. The weights give each point a vote, and the classification with the highest overall vote from the points that are in the k-nearest neighbors is the classification that is assigned to the value. The result of not using distance weighing is that all points have the same weight (1) even if one point is quite far away from the point itself.

Based on this, my prediction is that the results will be more accurate in the next section where we do use distance weighting (and also to some extent from normalization).

### 2.2 (10%) With Normalization
- Try it with k=3 without distance weighting but *with* normalization of input features.  You may use any reasonable normalization approach (e.g. standard min-max normalization between 0-1, z-transform, etc.)

In [ ]:
# Train/Predict with normalization
# W/Normalization, WOUT/Distance Weighting
X, y = load_magic_telescope()
X = normalize(X)

avg_train_acc = 0.0
avg_test_acc = 0.0
n_trials = 10
for i in range(n_trials):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
  knn = KNeighborsClassifier(n_neighbors=3, weights='uniform') # NOTE - uniform is default
  knn.fit(X_train, y_train)

  train_acc = knn.score(X_train, y_train)
  test_acc = knn.score(X_test, y_test)

  avg_train_acc += train_acc
  avg_test_acc += test_acc

  print("Trial:")
  print("   train score: ", str(train_acc))
  print("   test score: ", str(test_acc))

print("\nAverages:")
print("   avg train score: ", str(avg_train_acc / n_trials))
print("   avg test score: ", str(avg_test_acc / n_trials))

Trial:
   train score:  0.898593585699264
   test score:  0.8338590956887487
Trial:
   train score:  0.9014195583596214
   test score:  0.8349106203995794
Trial:
   train score:  0.9008280757097792
   test score:  0.830441640378549
Trial:
   train score:  0.9010252365930599
   test score:  0.8286014721345951
Trial:
   train score:  0.9024053627760252
   test score:  0.8301787592008412
Trial:
   train score:  0.9015509989484752
   test score:  0.8314931650893796
Trial:
   train score:  0.8999079915878023
   test score:  0.844111461619348
Trial:
   train score:  0.8996451104100947
   test score:  0.8378023133543638
Trial:
   train score:  0.8994479495268138
   test score:  0.833596214511041
Trial:
   train score:  0.9000394321766562
   test score:  0.8259726603575184

Averages:
   avg train score:  0.9004863301787592
   avg test score:  0.8330967402733964


*Discuss the results of using normalized data vs. unnormalized data*

As predicted, using normalized data did help a little bit. Both test and training set accuracies improved. Test set accuracy improved by about 1.5%, while training set accuracy improved by about 3%. While the difference isn't huge, it certainly makes it worth using. From what I've seen with using normalization here as well as with other algorithms in other labs, normalization provides a little more stability to the data.

For example, if most features have values between 5 and 10, while one feature has values between 5,000 and 10,000, the feature with larger value may skew the results from the other features. While many algorithms are able to compensate for this, it typically takes more time and often doesn't work perfectly. Therefore, normalizing the data is important to avoid these issues.

### 2.3 (10%) With Distance Weighting
- Try it with k=3 and with distance weighting *and* normalization

In [ ]:
#Train/Precdict with normalization and distance weighting
# W/Normalization, W/Distance Weighting
X, y = load_magic_telescope()
X = normalize(X)

avg_train_acc = 0.0
avg_test_acc = 0.0
n_trials = 10
for i in range(n_trials):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
  knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
  knn.fit(X_train, y_train)

  train_acc = knn.score(X_train, y_train)
  test_acc = knn.score(X_test, y_test)

  avg_train_acc += train_acc
  avg_test_acc += test_acc

  print("Trial:")
  print("   train score: ", str(train_acc))
  print("   test score: ", str(test_acc))

print("\nAverages:")
print("   avg train score: ", str(avg_train_acc / n_trials))
print("   avg test score: ", str(avg_test_acc / n_trials))

Trial:
   train score:  1.0
   test score:  0.8427970557308097
Trial:
   train score:  1.0
   test score:  0.830441640378549
Trial:
   train score:  1.0
   test score:  0.8288643533123028
Trial:
   train score:  1.0
   test score:  0.8351735015772871
Trial:
   train score:  1.0
   test score:  0.8259726603575184
Trial:
   train score:  1.0
   test score:  0.8280757097791798
Trial:
   train score:  1.0
   test score:  0.8293901156677181
Trial:
   train score:  1.0
   test score:  0.8291272344900105
Trial:
   train score:  1.0
   test score:  0.8359621451104101
Trial:
   train score:  1.0
   test score:  0.832807570977918

Averages:
   avg train score:  1.0
   avg test score:  0.8318611987381702


Comparison and Discussion

The default value for 'weights' in KNeighborsClassifier is 'uniform' which means it doesn't use distance weighting. Here, I change it to 'distance' which turns it on.

When distance weighting is used, the points that are further away from q have smaller weights. Voting occurs where each point gets a voting weight depending on its distance, and the classificaiton with the highest total vote is the classification that is assigned to q.

When looking at the results, the first thing to noticeis that the training accuracy was one for every trial. This makes sense, becuase the algorithm is weighting closer points higher. Since each point that we are testing was also used for training, our q will lie exactly on top of the cooreponding point used for training. Because of this, its weight will be very high, if not infinite, thus meaning that the classification for q will alway end up being the same as itself in from the when it was used in training. No other point could realistically overpower it.

Now, take a look at the test accuracies. It is interetsing to note that there was no improvement in test accuracy. While I was surprised at this, I think I understand the reasoning after thinking about it. I believe the problem is due to slight overfit. We are only using 3 as the k-value, which is a pretty low number. Since we are using distance weighting, we are much more likely to just take the value of the closest points even if they are just noise. We aren't letting the point look at enough general information. Not also that the accuracy didn't drop a significant amount. I think it is alright to keep using distance weighting, but what will really allow for better generalization here will be increasing k, which I will experiment with in the next section.

### 2.4 (10%) Different k Values
- Using your normalized data with distance weighting, create one graph with classification accuracy on the test set on the y-axis and k values on the x-axis.
- Use values of k from 1 to 15.  Use the same train/test split for each.

In [ ]:
# Calculate and Graph classification accuracy vs k values
# W/Normalization, W/Distance Weighting
X, y = load_magic_telescope()
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

headers = [i if i > 0 else "" for i in range(16)]
table = [["" if i > 0 else "Test acc" for i in range(16)]]

for k in range(1, 16):
  knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
  knn.fit(X_train, y_train)

  test_acc = knn.score(X_test, y_test)

  table[0][k] = test_acc

print(tabulate(table, headers=headers))

                 1         2         3         4         5         6         7         8       9        10        11        12        13        14        15
--------  --------  --------  --------  --------  --------  --------  --------  --------  ------  --------  --------  --------  --------  --------  --------
Test acc  0.804942  0.804942  0.831756  0.828076  0.845163  0.844111  0.845163  0.845952  0.8449  0.845426  0.844374  0.845689  0.845163  0.842797  0.845689


*Discussion*

Its very interesting to see how quite a bit of improvement was achieved from bumping up the k-value a few, then progress quickly stopped. At k=1 and 2, the results were identical. At 3, there was some improvement, at 4, results were slightly worse than three, and then at 5, results essentially peaked. Its important to note that at k=8, we did see the highest accuracy, though it was only by about 0.0005, and after that there was no more improvement.

I am strongly convinced that the value of k should depend on the shape of the data and how the test/real world data will vary from the training data. If data needs to be strongly generalized, it would be quite important to use a higher k value. However, if you know that the data is very rigid, fits specific patterns in a set way, and doesn't vary much from that, it may be better to use a smaller k. However, I believe that it is important to do a test like what I did above to experiment with different numbers for k and see which one actually yields the best results.

## 3 KNN Regression with normalization and distance weighting

Use the [sklean KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor) on the [housing price prediction](https://axon.cs.byu.edu/data/uci_regression/housing.arff) problem.  
### 3.1 (5%) Ethical Data
Note this data set has an example of an inappropriate input feature which we discussed.  State which feature is inappropriate and discuss why.

*Discuss the innapropriate feature*

The inappropriate feature in the dataset is `B` which is defined by the file as `1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town`. This feature is inappropriate for several reasons. For one, it can be used to discriminate. We don't want to predict house prices as being lower simply because more people of a single race live in one geographical location.

Additionally, it may not always be correct. While many populations that have a high percentage of black people may tend to have lower house prices, that is not always the case. Sometimes, it is exactly the opposite of that. If we use this feature, we may inappropriately lower the selling cost/evaluation of homes in high value areas.

In general, it is important to avoid having racial factors as data features, because by nature, it is generalizing a race even though it may not be fairly representative. Beyond this, it can be offensive and rude.

### 3.2 (15%) - KNN Regression
- Do random 80/20 train/test splits each time
- Run with k=3
- Print the score (coefficient of determination) and Mean Absolute Error (MAE) for the train and test set for the cases of
  - No input normalization and no distance weighting
  - Normalization and no distance weighting
  - Normalization and distance weighting
- Normalize inputs features where needed but do not normalize the output

In [ ]:
# Learn and experiment with housing price prediction data
# Score: Coefficient of determination
headers = ["Trial", "Train Score", "Train MAE", "Test Score", "Test MAE"]
table = []

trials = [
    {'normalize': False, 'weights': 'uniform', 'title': 'No input normalization and no distance weighting'},
    {'normalize': True, 'weights': 'uniform', 'title': 'Normalization and no distance weighting'},
    {'normalize': True, 'weights': 'distance', 'title': 'Normalization and distance weighting'}
]

for trial in trials:
  X, y = load_housing()

  if trial['normalize']:
    X = normalize(X)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

  knn = KNeighborsRegressor(n_neighbors=3, weights=trial['weights'])
  knn.fit(X_train, y_train)

  train_score = knn.score(X_train, y_train)
  test_score = knn.score(X_test, y_test)
  train_mae = mean_absolute_error(knn.predict(X_train), y_train)
  test_mae = mean_absolute_error(knn.predict(X_test), y_test)

  table.append([trial['title'], train_score, train_mae, test_score, test_mae])

print(tabulate(table, headers=headers))

Trial                                               Train Score    Train MAE    Test Score    Test MAE
------------------------------------------------  -------------  -----------  ------------  ----------
No input normalization and no distance weighting       0.793194      2.85759      0.434714     4.56928
Normalization and no distance weighting                0.892709      2.03399      0.812289     2.79542
Normalization and distance weighting                   1             0            0.824403     2.63068


*Discuss your results*

Here I perform 3 trials using the KNeighborsRegressor. As expected, results were best with normalization and distance weighting. As I explained above, the training score is 1, becasue distance weighting is used, and the weight for a point will be infinite or very high if it lines up with the point that is being scored. For the same reason, its mean absolute error (average absolute value of difference between actual and predicted) is 0.

Something that is interesting to note is that from the 2nd to 3rd trial, the MAE dropped by a more noticeable amount than the score increased when doing the same test with classification. As I explained above, I imagine it simple is due to the nuances in the dataset itself, and some improvement should be expected in cases where generalization is more important.

When you look at the dataset itself, the regression value ranges from about 8 to 50. When you compare the MAE values to this, they seem to be realistic. For example, on the 3rd trial, the MAE was 2.63. 2.63/50 = 5.26% so it appears to be within a threshold that is realistic. Its important to note that the score function returns the coefficient of determination, which is more complicated than the MAE. That is why the score wasn't just 1 - MAE.

### 3.3 (10%)  Different k Values
- Using housing with normalized data and distance weighting, create one graph with MAE on the test set on the y-axis and k values on the x-axis
- Use values of k from 1 to 15.  Use the same train/test split for each.

In [ ]:
# Learn and graph for different k values
X, y = load_housing()
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

headers = [i if i > 0 else "" for i in range(16)]
table = [["" if i > 0 else "Test MAE" for i in range(16)]]

for k in range(1, 16):
  knn = KNeighborsRegressor(n_neighbors=k, weights='distance')
  knn.fit(X_train, y_train)

  test_mae = mean_absolute_error(knn.predict(X_test), y_test)

  table[0][k] = test_mae

print(tabulate(table, headers=headers))

                1        2        3        4        5        6       7        8        9       10       11       12       13       14       15
--------  -------  -------  -------  -------  -------  -------  ------  -------  -------  -------  -------  -------  -------  -------  -------
Test MAE  3.12451  2.68805  2.81457  2.94946  3.02369  3.09449  3.2244  3.35564  3.39624  3.42056  3.45665  3.48168  3.48995  3.52985  3.51601


Discussion

Interestingly, the lowerst MAE was achieved with a k of 2, and it was 2.68. Comparing it to problem 3.2, it performed slightly words than 3.2 where k was 3 rather than 3, and distance weighting and normalization were also both used. This is a good case where the luck of the train/test split made a slight difference. Moreso than in classification, the MAE gradually dropped as k was increased past 2.

Also with all k values tried here (from 1 to 15), each still performed better than when not doing normalization or distance weighting (look at 3.2 above) where the training MAE was 4.57.

Again, this test was a good reminder that it is important to experiment with different values for hyperparameters, including k, with each problem you are solving with ML. There isn't always a perfect default, so it helps to find what works best for your problem. This is explained by the no-free-lunch theorem which states that and inductive bias that is good for some problems will probably be bad for equally as many problems.

## 4. (20%) KNN with nominal and real data

- Use the [lymph dataset](https://axon.cs.byu.edu/data/uci_class/lymph.arff)
- Use a 80/20 split of the data for the training/test set
- This dataset has both continuous and nominal attributes
- Implement a distance metric which uses Euclidean distance for continuous features and 0/1 distance for nominal. Hints:
    - Write your own distance function (e.g. mydist) and use clf = KNeighborsClassifier(metric=mydist)
    - Change the nominal features in the data set to integer values since KNeighborsClassifier expects numeric features. I used Label_Encoder on the nominal features.
    - Keep a list of which features are nominal which mydist can use to decide which distance measure to use
    - There was an occasional bug in SK version 1.3.0 ("Flags object has no attribute 'c_contiguous'") that went away when I upgraded to the lastest SK version 1.3.1
- Use your own choice for k and other parameters

In [ ]:
def my_dist(a: ndarray, b: ndarray, nominal_feature_indexes: set) -> float:
  """
  dist = euclidean_dist(continuous_features) + nominal_dist(nominal_features)
  """
  # separate into lists of nominal and continuous features
  a_nominal = []
  b_nominal = []
  a_cont = []
  b_cont = []
  for i in range(len(a)):
    if i in nominal_feature_indexes:
      a_nominal.append(a[i])
      b_nominal.append(b[i])
    else:
      a_cont.append(a[i])
      b_cont.append(b[i])

  # add distance for nominal and continuous features
  dist = 0.0
  dist += nominal_dist(a_nominal, b_nominal)
  dist += continuous_dist(a_cont, b_cont)

  return dist

def nominal_dist(a: List[float], b: list[float]) -> int:
  '''0/1 Distance (0 if they match, 1 otherwise)'''
  assert(len(a) == len(b))

  dist = 0
  for i in range(len(a)):
    if a[i] != b[i]:
      dist += 1

  return dist

def continuous_dist(a: List[float], b: list[float]) -> float:
  '''Euclidean Distance = sqrt(sum (1 to m) (x_i - y_i)^2)'''
  assert(len(a) == len(b))

  sum = 0.0
  for i in range(len(a)):
    sum += (a[i] - b[i]) ** 2

  return math.sqrt(sum)

In [ ]:
# Train/Predict lymph with your own distance metric
X, y = load_lymph()
lymph_nominal_feature_indexes = set([0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16])

# normalize continuous features
X = normalize_continuous_features(X, lymph_nominal_feature_indexes)

# label encode nominal features in X
for i in range(X.shape[1]):
  if i in lymph_nominal_feature_indexes:
    X[:, i] = LabelEncoder().fit_transform(X[:, i])
# label encode y
y = LabelEncoder().fit_transform(y)
headers = ["Trial", "Train Score", "Test Score", "Train Score", "Test Score"]
table = [["", "My Dist", "", "Default Dist", ""]]

avg_train_acc, avg_test_acc = 0.0, 0.0
n_trials = 10
for i in range(n_trials):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
  knn_my_dist = KNeighborsClassifier( # using my_dist
    n_neighbors=3,
    metric=my_dist,
    metric_params={'nominal_feature_indexes': lymph_nominal_feature_indexes}
  ).fit(X_train, y_train)

  knn_default_dist = KNeighborsClassifier( # using defualt dist
    n_neighbors=3,
  ).fit(X_train, y_train)

  train_acc_my_dist = knn_my_dist.score(X_train, y_train)
  test_acc_my_dist = knn_my_dist.score(X_test, y_test)
  train_acc_default_dist = knn_default_dist.score(X_train, y_train)
  test_acc_default_dist = knn_default_dist.score(X_test, y_test)

  table.append([i, train_acc_my_dist, test_acc_my_dist, train_acc_default_dist, test_acc_default_dist])

avg_train_my_dist = sum([table[row][1] for row in range(1, n_trials + 1)]) / n_trials
avg_test_my_dist = sum([table[row][2] for row in range(1, n_trials + 1)]) / n_trials
avg_train_defualt_dist = sum([table[row][3] for row in range(1, n_trials + 1)]) / n_trials
avg_test_defualt_dist = sum([table[row][4] for row in range(1, n_trials + 1)]) / n_trials

table.append(["Avg", avg_train_my_dist, avg_test_my_dist, avg_train_defualt_dist, avg_test_defualt_dist])

print(tabulate(table, headers=headers))

Trial    Train Score         Test Score          Train Score         Test Score
-------  ------------------  ------------------  ------------------  ------------------
         My Dist                                 Default Dist
0        0.923728813559322   0.9                 0.8898305084745762  0.7
1        0.9067796610169492  0.8                 0.8813559322033898  0.7333333333333333
2        0.9067796610169492  0.7666666666666667  0.8898305084745762  0.7
3        0.940677966101695   0.8666666666666667  0.8559322033898306  0.8
4        0.9067796610169492  0.9333333333333333  0.8898305084745762  0.7666666666666667
5        0.923728813559322   0.8333333333333334  0.9067796610169492  0.7
6        0.9152542372881356  0.8                 0.923728813559322   0.7
7        0.9152542372881356  0.8666666666666667  0.8898305084745762  0.7
8        0.9576271186440678  0.7666666666666667  0.9322033898305084  0.7
9        0.9067796610169492  0.8                 0.8898305084745762  0.7
Avg      0

*Explain your distance metric and discuss your results*

I am pretty happy with my results. To my metric function uses 0/1 distance for nominal features and euclidean distance for continuous features. The way I implemented this was by creating a distance function `my_dist`, which calls two helper functions, one for 0/1 and the other for euclidian distance. When trying to figure out the best way to do this, I was initially thinking of calling the euclidean distance function separately for each continuous feature, but I then realized that this wouldn't work right with the euclidean function, so at the beginning of `my_dist` I group all continuous features together, and all nonimal features together and send them into their respective functions all at the same time. For the nominal features, it doesn't make any difference doing them together or separate, because you would end up with the same resulting total either way.

I create a set of indexes of nominal features for the dataset which I pass into the distance function using the KNeighborsClassifier `metric_params` agrument which accepts an object containing parameters used in the metric function (in this case, `my_dist` which accepts a set `nominal_feature_indexes` as an argument).

I also realized it would be important to normalize all continuous features so that large distances wouldn't minimalize the distances from continuous features. I created a function `normalize_continuous_features` to do this, which also accepts `nominal_feature_indexes` to know which features to normalize (all those not appearing in `nominal_feature_indexes`).

My distance function actually outperformed the default one! My training set accuracy was about 3% higher than the default and the test set accuracy was about 11% better. One reason that my function did better is that the default distance function in Minkowski, and as shown in 1.1 above, using a different p-value could have improved its results. Additionally, my function knows how to treat nominal and continuous data differently, meaning that nominal data doesn't incorrectly make assumptions about distance that can't rightfully be made. For example, if 2 nominal features are 1 and 16, a continuous distance algorithm might treat them as relatively far apart, when in reality, the distance is no different than when comparing nominal values 1 and 2. Due to these reasons, my distance function performed pretty well!

If I had time to further tune my distance function, my next step would be to figure out a constant weight for both the nominal and continuous total distances. For example, the nominal distance may output a value of 1 while the continuous distance may be 2 when the more accurate value is one for both of them. Finding out which direction there is a skew will allow me to add a weight to one of them to make sure that both are balanced.

## 5. (Optional 15% extra credit) Code up your own KNN Learner
Below is a scaffold you could use if you want. Requirements for this task:
- Your model should support the methods shown in the example scaffold below
- Use Euclidean distance to decide closest neighbors
- Implement both the classification and regression versions
- Include optional distance weighting for both algorithms
- Run your algorithm on the magic telescope and housing data sets above and discuss and compare your results

*Discussion*

For my KNNClassifier and KNNRegressor, I decided to create a KNNBase class to avoid any code duplication. There, I create the main methods `__init__`, `fit`, `predict`, and `score`. `__init__` and `fit` are exactly the same for both classes. `predict` and `score` however, do have some differences. Do eliminate as much duplication as possible, I used the template method pattern for those two methods, and simply had `_predict` and `_score` as abstract methods for the child classes to implement.

I ended up using the same distance function as I created in step 4. This uses Euclidean distance for real features and 0/1 distance for categorical features.

I was happy that implementing `fit` was really simple. One of the benefits of KNN is that it doesn't actually require training. You simple have to save the datapoints. For the `score` function, in classification, I simply return the average accuracy, and for regression, I used the regression function from the slides: `f(x_q) = (sum (i = 1 to k) (w_i * f(X_i))) / (sum (i = 1 to k) (w_i))`, which works quite well.

For classification, the MagicTelescope dataset is so large and my algorithm isn't particularly efficient and the dataset is large, so I decided to not check training accuracy because the training set is so large that computation time would take a while. Looking at the results, my classifier scored almost exactly the same as Scikit Learn's. With both no distance weighting and distance weighting, both models scored about 83% and only varied by about 0.4%. I think this is likely to do with the fact that the dataset is quite large. The results demonstrate that my classifier works fairly well!

For the regressor, my results were close to Scikit-Learn's. For no distance weighting, I scored 1.98 for train MAE, while Scikit's scored 2.03, so I actually did better on that. For test MAE, mine was 3.26, and Scikit's was 2.79, meaning that Scikit's was quite a bit better than mine. Considering the fact that the housing values ranged from about 8 to about 50, 3.26 MAE really isn't that bad, and shows that my algorithm was working properly. When using inverse distance weighting, both models scored 0 MAE for training data. This is expected, because , once again, since training data was being used for testing, each point tested had a perfectly cooresponding neighbor in the data. When a neighbor is at the same spot (same feature values), the weight becomes infinity, so that feature's value is returned, meaning that there will NEVER be any error. For test values with distance weighting, mine had a MAE of 2.79, and Scikit's was 2.63, so mine was almost the same. I did notice that the train-test split did affect the results a good amount, so if I had more time, I would run several trials on my regressor and get averages.


In [74]:
class DataType(Enum):
  real = 1
  categorical = 2

class KNNBase(ABC, BaseEstimator,ClassifierMixin):
  def __init__(self, columntype: list[DataType], weight_type='inverse_distance', k=3):
    """
    Args:
      columntype for each column tells you if continues[real] or if nominal[categorical].
      weight_type: inverse_distance voting or if non distance weighting. Options = ["no_weight","inverse_distance"]
      k: the number of neighbors to look at
    """
    self.columntype = columntype
    self.weight_type = weight_type
    self.k = k

  def fit(self, data, labels):
    """ Fit the data; run the algorithm (for this lab really just saves the data :D)
    Args:
      X (array-like): A 2D numpy array with the training data, excluding targets
      y (array-like): A 2D numpy array with the training targets
    Returns:
      self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
    """
    assert(data.shape[0] == len(labels))

    self.X = data
    self.y = labels
    self.n_training_samples = len(labels)

    return self

  def predict(self, data):
    """ Predict all classes/regression values for a dataset X
    Args:
      X (array-like): A 2D numpy array with the training data, excluding targets
    Returns:
      array, shape (n_samples,)
        Predicted target values per element in X.
    """
    n = data.shape[0]

    res = [0] * n
    for i in range(n):
      res[i] = self._predict(data[i])

    return res

  #Returns the Mean score given input data and labels
  def score(self, X, y):
    """ Return accuracy of model on a given dataset. Must implement own score function.
    Args:
      X (array-like): A 2D numpy array with data, excluding targets
      y (array-like): A 2D numpy array with targets
    Returns:
      score : float
        Mean accuracy of self.predict(X) for Classification
        Mean absolute error (MAE) for regression
    """
    predictions = self.predict(X)

    return self._score(predictions, y)

  @abstractmethod
  def _predict(self, sample: ndarray):
    pass

  @abstractmethod
  def _score(self, predictions: list, y: ndarray):
    pass

  def _get_distances_to_all_samples(self, sample: ndarray) -> ndarray:
    '''Returns distance to each training sample, cooresponding by index'''
    distances = np.full(self.n_training_samples, np.inf)
    for i in range(self.n_training_samples):
      distances[i] = self._compute_distance(sample, self.X[i])

    return distances

  def _get_indexes_knn(self, sample: ndarray, distances: ndarray) -> ndarray:
    '''Returns: the indexes of the k-nearest neighbors'''
    assert(len(distances) == len(self.y))

    return np.argsort(distances)[:self.k]

  def _compute_distance_weight(self, dist: float) -> float:
    if dist == 0: return float('inf')

    return 1 / (dist ** 2)

  def _compute_distance(self, a: ndarray, b: ndarray) -> float:
    """
    dist = euclidean_dist(real_features) + nominal_dist(categorcal_features)

    Args:
      a: the features from a sample
      b: the features from another sample
    Returns:
      the distance from a to b
    """
    # separate into lists of nominal and continuous features
    a_nominal, b_nominal = [], []
    a_cont, b_cont = [], []
    for i in range(len(a)):
      if self.columntype[i] == DataType.categorical:
        a_nominal.append(a[i])
        b_nominal.append(b[i])
      else:
        a_cont.append(a[i])
        b_cont.append(b[i])

    # add distance for nominal and continuous features
    dist = self._compute_nominal_dist(a_nominal, b_nominal) + self._compute_continuous_dist(a_cont, b_cont)

    return dist

  def _compute_nominal_dist(self, a: List[float], b: list[float]) -> int:
    '''0/1 Distance (0 if they match, 1 otherwise)'''
    assert(len(a) == len(b))

    dist = 0
    for i in range(len(a)):
      if a[i] != b[i]:
        dist += 1

    return dist

  def _compute_continuous_dist(self, a: List[float], b: list[float]) -> float:
    '''Euclidean Distance = sqrt(sum (1 to m) (x_i - y_i)^2)'''
    assert(len(a) == len(b))

    sum = 0.0
    for i in range(len(a)):
      sum += (a[i] - b[i]) ** 2

    return math.sqrt(sum)

In [75]:
class KNNClassifier(KNNBase):
  def __init__(self, columntype: list[DataType], weight_type='inverse_distance', k=3):
    super().__init__(columntype, weight_type, k)

  def _predict(self, sample: ndarray):
    """
    Args:
      sample: an array containing all features from which to predict the class
    """
    distances = self._get_distances_to_all_samples(sample)
    indexes_knn = self._get_indexes_knn(sample, distances)

    if self.weight_type == 'inverse_distance': # inverse distance weighting
      clss = self._predict_inverse_distance_weighting(distances, indexes_knn)
    else: # no weighting
      clss = self._predict_no_weighting(distances, indexes_knn)

    return clss

  def _score(self, predictions: list, y: ndarray):
    matching = 0
    for i in range(len(y)):
      if y[i] == predictions[i]:
        matching += 1

    return matching / len(y)

  def _predict_inverse_distance_weighting(self, distances: ndarray, indexes_knn: ndarray):
    '''Returns the class with the highest total weight among knns'''
    # sum weight of each class from respective knns
    class_weights = defaultdict(float)
    for i in indexes_knn:
      clss = self.y[i]
      dist = distances[i]
      weight = self._compute_distance_weight(dist)

      class_weights[clss] += weight

    return max(class_weights, key=class_weights.get)

  def _predict_no_weighting(self, distances: ndarray, indexes_knn: ndarray):
    class_counts = defaultdict(int)
    for i in indexes_knn:
      clss = self.y[i]

      class_counts[clss] += 1

    return max(class_counts, key=class_counts.get)

In [80]:
class KNNRegressor(KNNBase):
  def __init__(self, columntype: list[DataType], weight_type='inverse_distance', k=3):
    super().__init__(columntype, weight_type, k)

  def _predict(self, sample: ndarray):
    """
    Args:
      sample: an array containing all features from which to predict the regression value
    """
    distances = self._get_distances_to_all_samples(sample)
    indexes_knn = self._get_indexes_knn(sample, distances)

    # get weights for each
    knn_weights = np.full(len(indexes_knn), 1.0)
    if self.weight_type == 'inverse_distance':
      # inverse distance weighting
      for i, knn_index in enumerate(indexes_knn):
        knn_weights[i] = self._compute_distance_weight(distances[knn_index])

    return self._compute_regression_value(indexes_knn, knn_weights)

  def _score(self, predictions: list, y: ndarray):
    '''Returns MAE (Mean absolute error)'''
    error = 0.0
    for i in range(len(y)):
      error += abs(y[i] - predictions[i])

    return error / len(y)

  def _compute_regression_value(self, indexes_knn: ndarray, knn_weights: ndarray):
    # f(x_q) = (sum (i = 1 to k) (w_i * f(X_i))) / (sum (i = 1 to k) (w_i))
    if float('inf') in knn_weights:
      # infinity weight (just return the y value of the sample with inf weight)
      inf_knn_i = np.where(knn_weights == float('inf'))
      inf_i = indexes_knn[inf_knn_i]

      return self.y[inf_i].item()

    top = sum(knn_weights[i] * self.y[knn_index] for i, knn_index in enumerate(indexes_knn))
    bottom = sum(knn_weights)

    return top / bottom

In [19]:
# TEST CLASSIFICATION: SIMPLE EXAMPLE
X = np.array([[.3, .8], [-.3, 1.6], [.9, 0], [1, 1]])
y = np.array(['A', 'B', 'B', 'A'])

columntype = [DataType.real, DataType.real]

knn = KNNClassifier(columntype=columntype, weight_type='no_weight', k=3)
knn.fit(X, y)

test_samples = np.array([[0.5, 0.2]])
print(knn.predict(test_samples))

['A']


In [88]:
# TEST CLASSIFICATION: MAGIC TELESCOPE
trials = [
    {'weight_type': 'no_weight', 'title': 'Magic Telescope: No distance weighting'},
    {'weight_type': 'inverse_distance', 'title': 'Magic Telescope: Inverse distance weighting'}
]

X, y = load_magic_telescope()
X = normalize(X)
columntype = [DataType.real, DataType.real, DataType.real, DataType.real, DataType.real, DataType.real, DataType.real, DataType.real, DataType.real, DataType.real]

for trial in trials:
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
  knn = KNNClassifier(columntype=columntype, weight_type=trial['weight_type'], k=3)
  knn.fit(X_train, y_train)

  test_acc = knn.score(X_test, y_test)

  print(trial['title'])
  print("   test score: ", str(test_acc))
  print()

Magic Telescope: No distance weighting
   test score:  0.8322818086225027

Magic Telescope: Inverse distance weighting
   test score:  0.8354363827549948



In [28]:
# TEST REGRESSION: SIMPLE EXAMPLE
X = np.array([[.3, .8], [-.3, 1.6], [.9, 0], [1, 1]])
y = np.array([.6, -.3, .8, 1.2])

columntype = [DataType.real, DataType.real]

knn = KNNRegressor(columntype=columntype, weight_type='no_weight', k=3)
knn.fit(X, y)

test_samples = np.array([[0.5, 0.2]])
print("prediction: ", str(knn.predict(test_samples)))

prediction:  [0.8666666666666666]


In [87]:
# TEST REGRESSION: HOUSING
trials = [
    {'weight_type': 'no_weight', 'title': 'Housing: No distance weighting'},
    {'weight_type': 'inverse_distance', 'title': 'Housing: Inverse distance weighting'}
]

X, y = load_housing()
X = normalize(X)
columntype = [DataType.real, DataType.real, DataType.real, DataType.categorical, DataType.real, DataType.real, DataType.real, DataType.real, DataType.real, DataType.real, DataType.real, DataType.real, DataType.real]

for trial in trials:
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
  knn = KNNRegressor(columntype=columntype, weight_type=trial['weight_type'], k=3)
  knn.fit(X_train, y_train)

  train_acc = knn.score(X_train, y_train)
  test_acc = knn.score(X_test, y_test)

  print(trial['title'])
  print("   train MAE: ", str(train_acc))
  print("   test MAE: ", str(test_acc))
  print()

Housing: No distance weighting
   train MAE:  1.9885313531353124
   test MAE:  3.267647058823527

Housing: Inverse distance weighting
   train MAE:  0.0
   test MAE:  2.6542582520705964

